In [1]:
using JuMP, Gurobi, CSV, DataFrames, JSON

data = CSV.read("test.csv",DataFrame)

Row,customers,vehicles,capacity,demand,edges,x_bar
,Int64,Int64,Int64,String,String,String
1,10,2,10,"[0, 1, 3, 2, 4, 1, 4, 2, 3, 2, 3]","[[1, 2], [2, 1], [1, 9], [9, 1], [1, 10], [10, 1], [1, 11], [11, 1], [2, 3], [3, 2], [3, 4], [4, 3], [4, 5], [5, 4], [5, 6], [6, 5], [6, 7], [7, 6], [6, 10], [10, 6], [7, 11], [11, 7], [10, 11], [11, 10], [7, 8], [8, 7], [8, 9], [9, 8], [1, 3], [3, 1], [4, 10], [10, 4], [1, 8], [8, 1], [7, 9], [9, 7]]","[1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]"


In [2]:
function exact_RCI_separation(k)
    
    edges = JSON.parse(data[k,"edges"])
    edges  = Vector{Vector{Int64}}(edges)

    sol_fraction = JSON.parse(data[k,"x_bar"])
    sol_fraction = Vector{Float64}(sol_fraction)

    demands = JSON.parse(data[k,"demand"])
    demands = Vector{Int64}(demands)

    s_hat = Vector{Vector{Int64}}()
    z = Vector{Float64}()
    rhs = Vector{Int64}()
    violation = Vector{Float64}()

    for M in 0:ceil(sum(demands[i] for i in 2:size(demands)[1])/data[k,"capacity"])-1

        model = Model(Gurobi.Optimizer)

        @variable(model,y[1:data[k,"customers"]+1],Bin)
        @variable(model,w[1:size(edges)[1]] >=0)

        @objective(model, Min, sum(sol_fraction[i]*w[i] for i in 1:size(edges)[1]))

        @constraint(model,constraint_4, y[1] == 0)
        @constraint(model, constraint3, sum(demands[i]*y[i] for i in 2:size(demands)[1]) >= M*data[k,"capacity"] + 1 )
        @constraint(model, constraint1[i in 1:size(edges)[1]], w[i]>=y[edges[i][1]] - y[edges[i][2]])

        JuMP.optimize!(model)

        
        s_hat_part = Vector{Int}()
        d = 0
        value_edge = Vector{Float64}()

        for i in 1:size(y)[1]
            if JuMP.value(y[i]) == 1.0
                push!(s_hat_part,i)
                d = d+ demands[i]
            end
        end

        push!(s_hat,s_hat_part)
        push!(z,JuMP.objective_value(model))
        push!(rhs, 2*ceil(d/data[k,"capacity"]) )
        push!(violation, 2*ceil(d/data[k,"capacity"]) - JuMP.objective_value(model))
    end

    output = [string(s_hat), string(z), string(rhs), string(violation)]

    return output
  
end

exact_RCI_separation (generic function with 1 method)

In [3]:
output_matrix = Vector{Vector{String}}()

for i in 1:size(data)[1]
    push!(output_matrix,exact_RCI_separation(i))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-26
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 38 rows, 47 columns and 119 nonzeros
Model fingerprint: 0x05cf4004
Variable types: 36 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 25 rows and 13 columns
Presolve time: 0.00s
Presolved: 13 rows, 34 columns, 58 nonzeros
Variable types: 24 continuous, 10 integer (10 binary)

Root relaxation: objective 3.750000e-01, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj

In [4]:
df = DataFrame(
    s_hat = getindex.(output_matrix, 1),
    z = getindex.(output_matrix, 2),
    rhs = getindex.(output_matrix, 3),
    violation = getindex.(output_matrix, 4)
)
CSV.write("output_test.csv", df)

"output_test.csv"